Set Up Flower Environment
# !pip install --quiet --upgrade flwr==1.5.0
# !pip install --quiet --upgrade tensorflow-cpu==2.14.1
# !pip install --quiet --upgrade scikit-learn
# !pip install --quiet --upgrade pandas
# !pip install --quiet --upgrade matplotlib
# !pip install --quiet --upgrade git+https://github.com/nepiskopos/py7zr.git

In [1]:
%pip install --quiet --upgrade flwr==1.5.0
%pip install --quiet --upgrade tensorflow
%pip install --quiet --upgrade tensorflow-cpu==2.14.1
%pip install --quiet --upgrade scikit-learn
%pip install --quiet --upgrade pandas
%pip install --quiet --upgrade matplotlib
%pip install --quiet --upgrade git+https://github.com/nepiskopos/py7zr.git

DEPRECATION: Loading egg at /opt/anaconda3/lib/python3.12/site-packages/gym-0.26.2-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /opt/anaconda3/lib/python3.12/site-packages/terminaltables-3.1.10-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /opt/anaconda3/lib/python3.12/site-packages/mlfromscratch-0.0.4-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /opt/anaconda3/lib/python3.12/site-packages/progressbar33-2.4-py3.12.egg is deprecated. pip 24

Dataset
This dataset is part of the DNP3 Intrusion Detection Dataset -- DOI: 10.21227/s7h0-b081 -- published by ITHACA – University of Western Macedonia.

In [2]:
import errno
import os
import py7zr
import re
import shutil
import sys
import time
import urllib


def reporthook(count, block_size, total_size):
    global start_time
    if count == 0:
        start_time = time.time()
        return
    duration = time.time() - start_time
    progress_size = int(count * block_size)
    speed = int(progress_size / (1024 * duration))
    percent = int(count * block_size * 100 / total_size)
    sys.stdout.write("\r...%d%%, %d MB, %d KB/s, %d seconds passed" %
                    (percent, progress_size / (1024 * 1024), speed, duration))
    sys.stdout.flush()

def download(url, filename):
    urllib.request.urlretrieve(url, filename, reporthook)

def extract(archive, target_dir):
    archive = py7zr.SevenZipFile(archive, mode='r')
    try:
        os.makedirs(target_dir)
        
        archive.extractall(path=target_dir)
        archive.close()
        
        print(f"Archive '{archive}' extracted successfully!")
    except OSError as e:
        if e.errno == errno.EEXIST:
            archive.extractall(path=target_dir)
            archive.close()
            
            print(f"Archive '{archive}' extracted successfully!")
        else:
            print(f"Archive '{archive}' failed to extract...")


base_data_dir = './datasets'

if not os.path.isdir(os.path.join(base_data_dir, 'Training_Testing_Balanced_CSV_Files')):
    filename = 'DNP3_Intrusion_Detection_Dataset_Final.7z'
    
    if not py7zr.is_7zfile(filename):
        url = ('https://zenodo.org/records/7348493/files/DNP3_Intrusion_Detection_Dataset_Final.7z?download=1')
        download(url, filename)
    
    extract(filename, base_data_dir)

    innerDirs = os.listdir(base_data_dir)
    for dir in innerDirs:
        if (re.search('2020*', dir)):
            shutil.rmtree(os.path.join(base_data_dir, dir))
    
    os.remove(os.path.join('.', filename))

In [3]:
import collections
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

pd.set_option('future.no_silent_downcasting', True)

%matplotlib inline

We must choose which dataset version to facilitate for the experimentation.

We must also choose 3 <= N >= 5 distinct federated learning workers.

For this example we use the CICFlowMeter dataset with 3 workers.

In [4]:
tp = 'cic'      # Type: dataset type (CICFlowMeter or Custom Python parser), choose between 'cic' and 'custom'
n_workers = 6   # N: number of federated workers, choose between 3 and 5

Perform some value checks

In [5]:
assert tp in ('cic', 'custom'), "Wrong dataset type, choose between 'cic' and 'custom'"
assert n_workers in (3, 6), "Wrong number of workers, choose between 3 and 5"

In [6]:
path = os.path.join(base_data_dir, 'Training_Testing_Balanced_CSV_Files')

if tp == 'cic':
    dataset = f'CICFlowMeter'
else:
    dataset = 'Custom_DNP3_Parser'

if 'CIC' in dataset:
    n = dataset.split('_')[-1]
    train_csv = os.path.join(os.path.join(path, dataset), f'CICFlowMeter_Training_Balanced.csv')
    test_csv = os.path.join(os.path.join(path, dataset), f'CICFlowMeter_Testing_Balanced.csv')
elif 'Custom' in dataset:
    n = dataset.split('_')
    train_csv = os.path.join(os.path.join(path, dataset), f'Custom_DNP3_Parser_Training_Balanced.csv')
    test_csv = os.path.join(os.path.join(path, dataset), f'Custom_DNP3_Parser_Testing_Balanced.csv')
else:
    raise Exception("Wrong dataset")

df_train = pd.read_csv(train_csv, sep=r'\s*,\s*', header=0, 
                       encoding='ascii', engine='python')
df_test = pd.read_csv(train_csv, sep=r'\s*,\s*', header=0, 
                      encoding='ascii', engine='python')

Let's perform some data exploration.

In [7]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5126 entries, 0 to 5125
Data columns (total 86 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0.1       5126 non-null   int64  
 1   Unnamed: 0         5126 non-null   int64  
 2   Flow ID            5126 non-null   object 
 3   Src IP             5126 non-null   object 
 4   Src Port           5126 non-null   int64  
 5   Dst IP             5126 non-null   object 
 6   Dst Port           5126 non-null   int64  
 7   Protocol           5126 non-null   int64  
 8   Timestamp          5126 non-null   object 
 9   Flow Duration      5126 non-null   int64  
 10  Tot Fwd Pkts       5126 non-null   int64  
 11  Tot Bwd Pkts       5126 non-null   int64  
 12  TotLen Fwd Pkts    5126 non-null   float64
 13  TotLen Bwd Pkts    5126 non-null   float64
 14  Fwd Pkt Len Max    5126 non-null   float64
 15  Fwd Pkt Len Min    5126 non-null   float64
 16  Fwd Pkt Len Mean   5126 

In [8]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5126 entries, 0 to 5125
Data columns (total 86 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0.1       5126 non-null   int64  
 1   Unnamed: 0         5126 non-null   int64  
 2   Flow ID            5126 non-null   object 
 3   Src IP             5126 non-null   object 
 4   Src Port           5126 non-null   int64  
 5   Dst IP             5126 non-null   object 
 6   Dst Port           5126 non-null   int64  
 7   Protocol           5126 non-null   int64  
 8   Timestamp          5126 non-null   object 
 9   Flow Duration      5126 non-null   int64  
 10  Tot Fwd Pkts       5126 non-null   int64  
 11  Tot Bwd Pkts       5126 non-null   int64  
 12  TotLen Fwd Pkts    5126 non-null   float64
 13  TotLen Bwd Pkts    5126 non-null   float64
 14  Fwd Pkt Len Max    5126 non-null   float64
 15  Fwd Pkt Len Min    5126 non-null   float64
 16  Fwd Pkt Len Mean   5126 

As a preprocessing step, we transorm the "Label" feature from string to categorical and then to numeric.

In [9]:
df_train['Label'] = df_train['Label'].str.lower()
df_test['Label'] = df_test['Label'].str.lower()

unique_labels = list(df_train.Label.astype('category').unique())
unique_codes = list(df_train.Label.astype('category').cat.codes.unique())

mapping = dict(zip(unique_labels, unique_codes))
mapping_inv = dict(zip(unique_codes, unique_labels))

In [10]:
mapping

{'init_data': 5,
 'disable_unsolicited': 2,
 'warm_restart': 10,
 'mitm_dos': 6,
 'arp_poisoning': 0,
 'normal': 7,
 'dnp3_enumerate': 3,
 'cold_restart': 1,
 'dnp3_info': 4,
 'replay': 8,
 'stop_app': 9}

In [11]:
df_train['Label'] = df_train['Label'].astype('category').cat.rename_categories(mapping)
df_test['Label'] = df_test['Label'].astype('category').cat.rename_categories(mapping)

Now, we remove the features which do not assist with training a ML model.

In [12]:
train = df_train.iloc[:,9:]
test = df_test.iloc[:,9:]

We rename the "Label" column to "y".

In [13]:
train.rename(columns={"Label": "y"}, inplace=True, errors="raise")
test.rename(columns={"Label": "y"}, inplace=True, errors="raise")

We drop Inf and NaN values both in the training and in test feature data.

In [14]:
# Replace Inf values with NaN
train.replace([np.inf, -np.inf], np.nan, inplace=True)
test.replace([np.inf, -np.inf], np.nan, inplace=True)

# Drop rows with NaN values
train.dropna(inplace=True)
test.dropna(inplace=True)

We store the modified datasets into new local CSV files for future use and model evaluation.

In [15]:
directory = os.path.join(base_data_dir, 'federated_datasets')

try:
    os.makedirs(directory)
    train.to_csv(os.path.join(directory, 'train_data.csv'), index=False)
    test.to_csv(os.path.join(directory, 'test_data.csv'), index=False)
except OSError as e:
    if e.errno != errno.EEXIST:
        raise
    else:   
        train.to_csv(os.path.join(directory, 'train_data.csv'), index=False)
        test.to_csv(os.path.join(directory, 'test_data.csv'), index=False)

Each worker will be assigned a subset of the dataset with [M/N] data points.

Each worker data is stored as a seperate CSV file with the proper naming scheme.

In [16]:
n_samples = int(train.shape[0] / n_workers)

assert type(n_workers) == int, "Non-int number of workers"
assert n_workers >= 6 and n_workers <= df_train.shape[0], "At least 3 workers and at most as many workers as the number of samples are allowed"
assert n_samples > 0, "Each worker must be assigned at least one data point"

In [17]:
client_data = []
train_copy = train.copy()

for i in range(n_workers):
    sample = train_copy.sample(n=n_samples)
    sample.to_csv(os.path.join(directory, f'client_train_data_{i+1}.csv'), index=False)
    train_copy.drop(index=sample.index, inplace=True)
    client_data.append(sample)

Let's check the number of points in each straggler's training data.

In [18]:
for i in range(n_workers):
    print(f"Worker {i+1} training data contains {len(client_data[i])} points")

Worker 1 training data contains 854 points
Worker 2 training data contains 854 points
Worker 3 training data contains 854 points
Worker 4 training data contains 854 points
Worker 5 training data contains 854 points
Worker 6 training data contains 854 points


Let's perform some statistical analysis on the difference in the number of samples for each digit in each client dataset.

In [19]:
# Number of examples per layer for a sample of clients
fig = plt.figure(figsize=(20, 7))
fig.suptitle('Label Counts for a Sample of Worker Data')
fig.tight_layout()

for i in range(n_workers):
    m = 0
    plot_data = collections.defaultdict(list)
    sample = client_data[i]
    for label in sample['y']:
        # Append counts individually per label to make plots
        # more colorful instead of one color per plot.
        plot_data[label].append(label)
        m = max(m, len(plot_data[label]))
    
    n_cols = n_workers if n_workers < 5 else 5
    xlim = [0, m+(20-m%20)]
    ylim = [min(unique_codes)-1, max(unique_codes)+1]
    yticks = list(range(min(unique_codes), max(unique_codes)+1))
    yticks_labels = [mapping_inv[k] for k in range(0, max(unique_codes)+1)]
    
    plt.subplot(int(n_workers / 6)+1, n_cols, i+1)
    plt.subplots_adjust(wspace=0.3)
    plt.title('Worker {}'.format(i+1))
    plt.xlabel('#points')
    plt.xlim(xlim)
    plt.ylabel('Label')
    plt.ylim(ylim)
    plt.yticks(yticks, labels=yticks_labels)
    
    # plot values on top of bars
    for key in plot_data:
        if len(plot_data[key]) > 0:
            plt.text(len(plot_data[key])+8, int(key)-0.1, str(len(plot_data[key])), ha='center')
    
    for j in range(min(unique_codes),max(unique_codes)+1):
        plt.hist(
            plot_data[j],
            density=False,
            bins=[k-0.5 for k in range(min(unique_codes),max(unique_codes)+2)],
            orientation='horizontal'
        )

In [20]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical

from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler

In [21]:

scaler = StandardScaler()

In [22]:
X_train = scaler.fit_transform(train.drop(['y'], axis=1).to_numpy())
y_train = train['y'].to_numpy()
X_test = scaler.transform(test.drop(['y'], axis=1).to_numpy())
y_test = test['y'].to_numpy()

In [23]:
y_cat_train = to_categorical(y_train)
y_cat_test = to_categorical(y_test)

In [24]:
model = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(units=50, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=y_cat_test.shape[1], activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


In [25]:
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)

In [26]:
model.fit(
    x=X_train, 
    y=y_cat_train, 
    epochs=300,
    validation_data=(X_test, y_cat_test),
    verbose=1,
    batch_size=64,
    callbacks=[early_stop]
)

Epoch 1/300
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.3282 - loss: 1.9253 - val_accuracy: 0.5900 - val_loss: 1.1782
Epoch 2/300
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 807us/step - accuracy: 0.5050 - loss: 1.1935 - val_accuracy: 0.5762 - val_loss: 1.0587
Epoch 3/300
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 813us/step - accuracy: 0.5427 - loss: 1.0845 - val_accuracy: 0.6892 - val_loss: 0.9963
Epoch 4/300
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 805us/step - accuracy: 0.5551 - loss: 1.0400 - val_accuracy: 0.6892 - val_loss: 0.9403
Epoch 5/300
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 830us/step - accuracy: 0.5922 - loss: 0.9643 - val_accuracy: 0.6109 - val_loss: 0.8853
Epoch 6/300
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 807us/step - accuracy: 0.6237 - loss: 0.8955 - val_accuracy: 0.6423 - val_loss: 0.8349
Epoch 7/300
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 814us/step - accuracy: 0.6128 - loss: 0.8692 - val_accuracy: 0.6909 - val_loss: 0.7905
Epoch 8/300
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 784us/step - accuracy: 0.6246 - loss: 0.8259 - val_accura

In [27]:

model_loss = pd.DataFrame(model.history.history)
model_loss[['loss','val_loss']].plot()

<Axes: >

In [28]:

print("Loss, Accuracy: ", model.evaluate(X_test, y_cat_test))

161/161 ━━━━━━━━━━━━━━━━━━━━ 0s 580us/step - accuracy: 0.7754 - loss: 0.4411
Loss, Accuracy:  [0.44721555709838867, 0.7791219353675842]


In [29]:
print("F1-score: ", f1_score(y_test, np.argmax(model.predict(X_test), axis=1), average='weighted'))

161/161 ━━━━━━━━━━━━━━━━━━━━ 0s 353us/step
F1-score:  0.7436798412801852
